# DISFA

## Importing libraries and setting folders

Libraries

In [15]:
import os, sys, random, glob, argparse, math, gc

In [16]:
import cv2
import dlib
import imutils
from imutils import face_utils
import matplotlib
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import data, exposure

In [19]:
import numpy as np
import pandas as pd
from bcolz import carray

In [18]:
from tqdm import tqdm
from time import sleep

Folders

In [14]:
folder_DISFA_data = "/media/amogh/Stuff/CMU/datasets/DISFA_data/"
folder_DISFA_FAU = "/media/amogh/Stuff/CMU/datasets/DISFA_data/ActionUnit_Labels/"
folder_DISFA_FAU_summary = "DISFA_FAUs/"

## Helper functions

### Getting a dictionary with positives and negatives for each subject and frame

In [216]:
# returns a dictionary in the form: {'SN001':{'positives': [1,2,3],'negatives':[4,5,6,7] }}
# ie corresponding to each subject a dictionary which contains list frame nos which are positives and 
def getDISFAFramesDictionary(folder_DISFA_FAU_summary, fau_no, fau_thresh):
    df_fau = pd.read_csv(folder_DISFA_FAU_summary + "{}/".format(fau_thresh) + "FAU{}.csv".format(fau_no))
    df_positives = df_fau.filter(regex="^((?!neg).)*$",axis=1)
    df_negatives = df_fau.filter(like="neg",axis=1) 
    list_subjects = df_positives.columns.values
    fau_dict = {}
    for subj in list_subjects:
        fau_dict[subj] = {'positives':[], 'negatives':[]}
        fau_dict[subj]['positives'] = [f for f in df_positives[subj].values if not math.isnan(f)]
        fau_dict[subj]['negatives'] = [f for f in df_negatives["{}_neg".format(subj)].values if not math.isnan(f)]
    return fau_dict

### Get test and train folds of the data

In [238]:
# returns a dictionary with keys as fold_0,fold_1,...,test
# make sure number of folds exactly divide the train subjects
def getTrainTestFolds (fau_dict, no_folds, no_test_subjects):
    list_subjects = fau_dict.keys()
    no_train_subjects = len(list_subjects) - no_test_subjects
    random.shuffle(list_subjects)
    test_subjects = list_subjects[-no_test_subjects:]
    train_subjects = list_subjects[:-no_test_subjects]
    dict_folds = {'test':{}}
    # putting train and test subjects in new dictionary
    for subj in test_subjects:
        dict_folds['test'][subj] = fau_dict[subj]
    fold_size = no_train_subjects / no_folds
#     fold_size_remainder = no_train_subjects % no_folds
    for fold_no in range(no_folds):
        fold_subjects = train_subjects[fold_no*fold_size : fold_no*fold_size+fold_size]
        dict_folds ['fold_{}'.format(fold_no)]={}
        for sub in fold_subjects:
            dict_folds ['fold_{}'.format(fold_no)] [sub] = fau_dict [sub]
    return dict_folds

### Crop and save images

##### Function for cropping given an image path 

In [ ]:
def similarityTransform(inPoints, outPoints) :
    s60 = math.sin(60*math.pi/180);
    c60 = math.cos(60*math.pi/180);  
  
    inPts = np.copy(inPoints).tolist();
    outPts = np.copy(outPoints).tolist();
    
    xin = c60*(inPts[0][0] - inPts[1][0]) - s60*(inPts[0][1] - inPts[1][1]) + inPts[1][0];
    yin = s60*(inPts[0][0] - inPts[1][0]) + c60*(inPts[0][1] - inPts[1][1]) + inPts[1][1];
    
    inPts.append([np.int(xin), np.int(yin)]);
    
    xout = c60*(outPts[0][0] - outPts[1][0]) - s60*(outPts[0][1] - outPts[1][1]) + outPts[1][0];
    yout = s60*(outPts[0][0] - outPts[1][0]) + c60*(outPts[0][1] - outPts[1][1]) + outPts[1][1];
    
    outPts.append([np.int(xout), np.int(yout)]);
    
    tform = cv2.estimateRigidTransform(np.array([inPts]), np.array([outPts]), False);
    
    return tform;

In [ ]:
#new function, doesnt write landmarks every single time
def detectAndaligncrop(impath, detector, predictor, shape_predictor_filename):
    image=cv2.imread(impath)
    image_float=np.float32(image)/255.0
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)
    #initialising images and allPoints arrays
    allPoints=[]
    for (i, rect) in enumerate(rects):
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        points=[]
        for (x,y) in shape:
            points.append((x,y))
        allPoints.append(points)
    images=[image_float]
    #computation
    w=112
    h=112
    eyecornerDst = [ (np.int(0.3 * w ), np.int(h / 3)), (np.int(0.7 * w ), np.int(h / 3)) ];
    imagesNorm = [];
    pointsNorm = [];
    #     print allPoints[0]
    # Add boundary points for delaunay triangulation
    boundaryPts = np.array([(0,0), (w/2,0), (w-1,0), (w-1,h/2), ( w-1, h-1 ), ( w/2, h-1 ), (0, h-1), (0,h/2) ]);
    n = len(allPoints[0]);
    numImages = len(images)
    for i in xrange(0, numImages):
        points1 = allPoints[i];
        # Corners of the eye in input image
        eyecornerSrc  = [ allPoints[i][36], allPoints[i][45] ] ;
        # Compute similarity transform
        tform = similarityTransform(eyecornerSrc, eyecornerDst);
        # Apply similarity transformation
        img = cv2.warpAffine(images[i], tform, (w,h));
    #         print("debug im type shape max mean min ", img.dtype,img.shape,np.max(img),np.mean(img),np.min(img))
    #         plt.imshow(img)
        # Apply similarity transform on points
        points2 = np.reshape(np.array(points1), (68,1,2));        
        points = cv2.transform(points2, tform);
        points = np.float32(np.reshape(points, (68, 2)));
        pointsNorm.append(points);
        imagesNorm.append(img);
    #     print (pointsNorm[0])
    #     plt.imshow(imagesNorm[0]) 
    # Output image
    output=imagesNorm[0]
    rgb_image=cv2.cvtColor(output,cv2.COLOR_BGR2RGB)
    return rgb_image, pointsNorm[0]

##### Crop and save function

In [281]:
def cropAndSave (fau_no, thresh, dict_folds, folder_DISFA_data, cropping_function_name, function_dict, boolSave=True):
    folder_cropped_images = folder_DISFA_data + "/features/cropped_images/"
    folder_dest = folder_cropped_images +  "/{}/{}/".format(thresh,cropping_function_name)
    # initialize dlib's face detector (HOG-based) and then create
    # the facial landmark predictor
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    if not os.path.exists(folder_dest):
        os.makedirs(folder_dest)
    for fold in dict_folds.keys():
        print(dict_folds.keys())
        for subj in dict_folds[fold]:
            print(subj)
            for category in dict_folds[fold][subj]:
                print (category)
                for frame in dict_folds[fold][subj][category]:
                    im_path = folder_DISFA_data + "Videos_RightCamera/RightVideo{}/{}.jpeg".format(subj,int(frame))
                    if os.path.exists(im_path):
                        try:
                            
                        
cropAndSave(2, 3, getTrainTestFolds(getDISFAFramesDictionary(folder_DISFA_FAU_summary,2,3),5,2), folder_DISFA_data, 1,2)

['fold_4', 'fold_2', 'fold_3', 'fold_0', 'fold_1', 'test']
SN017
positives
negatives
SN023
positives
negatives
SN003
positives
negatives
SN026
positives
negatives
SN032
positives
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/931.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/932.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/933.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/934.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/935.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/936.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/937.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032

('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/1964.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/1965.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/1966.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/1967.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/1968.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/1969.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/1970.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/1971.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/1972.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DIS

('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/4024.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/4025.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/4026.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/4027.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/4028.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/4029.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/4030.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/4031.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN032/4032.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DIS

SN028
positives
negatives
SN027
positives
negatives
SN024
positives
negatives
SN009
positives
negatives
['fold_4', 'fold_2', 'fold_3', 'fold_0', 'fold_1', 'test']
SN031
positives
negatives
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/1.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/2.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/3.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/4.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/5.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/6.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/7.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/8.j

('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/2071.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/2072.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/2073.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/2074.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/2075.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/2076.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/2077.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/2078.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/2079.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DIS

('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/4185.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/4186.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/4187.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/4188.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/4189.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/4190.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/4191.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/4192.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN031/4193.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DIS

SN004
positives
negatives
SN012
positives
negatives
SN025
positives
negatives
['fold_4', 'fold_2', 'fold_3', 'fold_0', 'fold_1', 'test']
SN030
positives
negatives
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN030/4.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN030/8.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN030/28.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN030/33.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN030/34.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN030/35.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN030/36.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN030/37.jpeg', 'not present')

('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN030/2859.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN030/2860.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN030/2861.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN030/2862.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN030/2863.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN030/2864.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN030/2865.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN030/2866.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN030/2867.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DIS

('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN030/4839.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN030/4840.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN030/4841.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN030/4842.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN030/4843.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN030/4844.jpeg', 'not present')
('/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN030/4845.jpeg', 'not present')
SN002
positives
negatives
SN013
positives
negatives
SN005
positives
negatives
SN018
positives
negatives
['fold_4', 'fold_2', 'fold_3', 'fold_0', 'fold_1', 'test']
SN029
positives
negatives
SN021
positives
negatives
SN011
positives
negatives
SN006
posi

## Main Function:

In [10]:
def trainDISFA (fau_no, train_no, fau_thresh, test_subjects_no, boolGetLists=False, boolCalcFeatures=False, boolCrossValidation=True, ):
    if boolGetLists:
        getDISFALists

SyntaxError: invalid syntax (<ipython-input-10-3319ea1e9eb8>, line 1)